In [1]:
from imblearn.over_sampling import SMOTE
import numpy as np, pandas as pd
from matplotlib import pyplot
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy import log
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, accuracy_score, precision_score, recall_score, precision_recall_curve
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance


In [3]:
df = pd.read_csv("C:/Users/szymb/Desktop/Blazej/ML/spotify_preprocessed.csv")

## Splitting dataset - train, validation, test

In [4]:
def split_dataset(df, sample=1):
    df = df.sample(int(sample * df.shape[0]))
    train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

    print(f" Train: {train.shape}\n", f"Validate: {validate.shape}\n", f"Test: {test.shape}\n")

    Xtrain = train.loc[:, df.columns != 'label']

    Xvalidate = validate.loc[:, df.columns != 'label']

    Xtest = test.loc[:, df.columns != 'label']


    ytrain = train.label.values.ravel()

    yvalidate = validate.label.values.ravel()

    ytest = test.label.values.ravel()

    return Xtrain, Xvalidate, Xtest, ytrain, yvalidate, ytest

In [5]:
trainX, validateX, testX, trainy, validatey, testy = split_dataset(df,sample=1)

 Train: (345369, 19)
 Validate: (115123, 19)
 Test: (115123, 19)



Although the opposite is unlikely, it is worth making sure that the distribution of the variables labeled as 1 is comparable in all subsets

In [6]:
round(trainy.sum()/len(trainy),4)

0.1006

In [7]:
round(validatey.sum()/len(validatey),4)

0.1005

In [8]:
round(testy.sum()/len(testy),4)

0.0991

## Data normalization

Although data normalization is not necessary for artificial neural networks, and especially in 'MLP'. 
This is due to the fact that weights and biases can always be appropriately 'shifted' and scaled up or down. 
However, there are practical reasons for this approach - especially speeding up the calculation and reducing the risk of 
getting stuck at local minima.

In [9]:
s = StandardScaler()
trainX = s.fit_transform(trainX)
validateX = s.transform(validateX)
testX = s.transform(testX)

## Model preparation

Since the ratio of the dependent variables is skewed, additional metrics than accuracy are necessary.

In order to compare the results of the neural network with other models, we decided that the best metric would be f1. However, it is not built into the library, so custom functions are required.

In [11]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint 
class Metrics(Callback):
    def __init__(self, validation):   
        super(Metrics, self).__init__()
        self.validation = validation    
            
        print('validation shape', len(self.validation[0]))
        
    def on_train_begin(self, logs={}):        
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        val_targ = self.validation[1]   
        val_predict = (np.asarray(self.model.predict(self.validation[0]))).round()        
    
        val_f1 = f1_score(val_targ, val_predict)
        val_recall = recall_score(val_targ, val_predict)         
        val_precision = precision_score(val_targ, val_predict)
        
        self.val_f1s.append(round(val_f1, 6))
        self.val_recalls.append(round(val_recall, 6))
        self.val_precisions.append(round(val_precision, 6))
 
        print(f' — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')

In [12]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [13]:
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [14]:
def f1_weighted_loss(true, pred): #shapes (batch, 4)
    ground_positives = K.sum(true, axis=0) + K.epsilon()
    pred_positives = K.sum(pred, axis=0) + K.epsilon()
    true_positives = K.sum(true * pred, axis=0) + K.epsilon() 
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)
    return 1 - weighted_f1

In [ ]:
def f1_weighted_metric(true, pred):
    predLabels = K.argmax(pred, axis=-1)
    pred = K.one_hot(predLabels, 4) 
    ground_positives = K.sum(true, axis=0) + K.epsilon()
    pred_positives = K.sum(pred, axis=0) + K.epsilon()
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)
    return weighted_f1

Relu function is used for hidden layer as it has the best performance for that architecture (and architecture is best for problem type).

And sigmoid function is used for output layer, as the output is binar

In [ ]:
# This function is intended to help deciding on the architecture of the network

def prepare_model (output_bias = None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = Sequential()
    model.add(Dense(40, input_dim=18, activation='relu'))
    model.add(Dense(20, input_dim=40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=[f1,
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall'),
                           keras.metrics.AUC(name='auc'),
                           keras.metrics.AUC(name='prc', curve='PR'),])
    return model

In [ ]:
model = prepare_model()
model.summary()

In [ ]:
model.predict(trainX[:50])

In [ ]:
results = model.evaluate(trainX, trainy, batch_size=50, verbose=0)
print("Loss of the library built-in initialization: " + str(round(results[0],5)))

As the 'loss' value, although obviously too large to be accepted in the final model, does not exceed .9 (in the iterations so far).

Therefore, it will not be necessary to manually modify the biases before initializing the model - the first few epochs are enough for the model to learn that the ratio of the dependent variables is not comparable (values labeled as 0 and 1)

In [ ]:
num_positives = df['label'].sum()
print(num_positives)
num_negative = len(df['label'])-num_positives
initial_bias = np.log([num_positives/num_negative])
initial_bias

In [ ]:
model = prepare_model(output_bias=initial_bias)
model.predict(trainX[:50])

In [ ]:
results = model.evaluate(trainX, trainy, batch_size=50, verbose=0)
print("Loss of the initialization with self output-bias initialization: " + str(round(results[0],5)))

To prevent overfitting, early stopping is used.

## Grid search - checking best suited number of nods in hidden layer

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.2)
def grid_search(
    trainX, trainy, validateX, validatey, testX, testy, max_hidden_layers, nods
):
    sm = SMOTE(random_state=42, n_jobs=3)
    trainX, trainy = sm.fit_resample(trainX, trainy)
    trainX, trainy, validateX, validatey, testX, testy = (
        trainX.astype(np.float32),
        trainy.astype(np.float32),
        validateX.astype(np.float32),
        validatey.astype(np.float32),
        testX.astype(np.float32),
        testy.astype(np.float32),
    )
    best_model = [-1, 0]
    if max_hidden_layers == 1:
        for nod1 in nods:

            def prepare_model(nod1=nod1):
                model = Sequential()
                model.add(Dense(nod1, input_dim=18, activation="relu"))
                model.add(Dense(1, activation="sigmoid"))
                model.compile(
                    loss=f1_loss,
                    optimizer="adam",
                    metrics=[
                        f1,
                        keras.metrics.Precision(name="precision"),
                        keras.metrics.Recall(name="recall"),
                        keras.metrics.AUC(name="auc"),
                        keras.metrics.AUC(name="prc", curve="PR"),
                    ],
                )
                return model

            model = prepare_model()
            history = (
                model.fit(
                    trainX,
                    trainy,
                    callbacks=[Metrics(validation=(validateX, validatey))],
                    epochs=100,
                    batch_size=100,
                    verbose=1,
                ),
            )
            test_results = model.evaluate(testX, testy, verbose=0)
            if test_results[1] > best_model[0]:
                best_model = [test_results[1], nod1]
            print(best_model)
    if max_hidden_layers == 2:
        for nod1 in nods:
            for nod2 in nods:

                def prepare_model(nod1=nod1, nod2=nod2):
                    model = Sequential()
                    model.add(Dense(nod1, input_dim=18, activation="relu"))
                    model.add(Dense(nod2, activation="relu"))
                    model.add(Dense(1, activation="sigmoid"))
                    model.compile(
                        loss=f1_loss,
                        optimizer="adam",
                        metrics=[
                            f1,
                            keras.metrics.Precision(name="precision"),
                            keras.metrics.Recall(name="recall"),
                            keras.metrics.AUC(name="auc"),
                            keras.metrics.AUC(name="prc", curve="PR"),
                        ],
                    )
                    return model

                model = prepare_model()
                history = (
                    model.fit(
                        trainX,
                        trainy,
                        callbacks=[Metrics(validation=(validateX, validatey))],
                        epochs=100,
                        batch_size=100,
                        verbose=1,
                    ),
                )
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    verbose=1,
                    patience=20,
                    mode="max",
                    restore_best_weights=True,
                ),
                test_results = model.evaluate(testX, testy, verbose=0)
                if test_results[1] > best_model[0]:
                    best_model = [test_results[1], nod1, nod2]
                print(best_model)
        
    if max_hidden_layers == 4:
        for nod1 in nods:
            for nod2 in nods:
                for nod3 in nods:
                    for nod4 in nods:

                        def prepare_model(nod1=nod1, nod2=nod2, nod3=nod3,nod4=nod4):
                            
                            model = Sequential()
                            model.add(Dense(nod1, input_dim=18, activation="relu"))
                            model.add(Dense(nod2, activation="relu"))
                            model.add(Dense(nod3, activation="relu"))
                            model.add(Dense(nod4, activation="relu"))
                            model.add(Dense(1, activation="sigmoid"))
                            model.compile(
                                loss=f1_loss,
                                optimizer="adam",
                                metrics=[
                                    f1,
                                    keras.metrics.Precision(name="precision"),
                                    keras.metrics.Recall(name="recall"),
                                    keras.metrics.AUC(name="auc"),
                                    keras.metrics.AUC(name="prc", curve="PR"),
                                ],
                            )
                            return model

                        model = prepare_model()
                        history = (
                            model.fit(
                                trainX,
                                trainy,
                                callbacks=[Metrics(validation=(validateX, validatey))],
                                epochs=100,
                                batch_size=100,
                                verbose=1,
                            ),
                        )
                        tf.keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            verbose=1,
                            patience=20,
                            mode="max",
                            restore_best_weights=True,
                        ),
                        test_results = model.evaluate(testX, testy, verbose=0)
                        if test_results[1] > best_model[0]:
                            best_model = [test_results[1], nod1, nod2,nod3,nod4]
                        print(best_model)
    return best_model


In [ ]:
nods = [4,5,8,10,12,15,20,30,40]
grid_search(trainX,trainy,testX,testy,validateX,validatey, 4 , nods)

In [ ]:
nods = [1,2,3,4,5,10,15,20,25,30,35,40,45,50,55,60]
grid_search(trainX,trainy,testX,testy,validateX,validatey, 1 , nods)

In [ ]:
nods = [2,3,4,5,10,15,20,25,30,35,40,45,50,55,60]
grid_search(trainX,trainy,testX,testy,validateX,validatey, 2 , nods)

## Running model with best performacne (regarding f1 score)

In [41]:

sm = SMOTE(random_state=42, n_jobs=3)
trainX, trainy = sm.fit_resample(trainX, trainy)
trainX, trainy, validateX, validatey, testX, testy = (
    trainX.astype(np.float32),
    trainy.astype(np.float32),
    validateX.astype(np.float32),
    validatey.astype(np.float32),
    testX.astype(np.float32),
    testy.astype(np.float32),)
def prepare_model(nod1=40, nod2=30, nod3=20,nod4=10):
                            model = Sequential()
                            model.add(Dense(nod1, input_dim=18, activation="relu"))
                            model.add(Dense(nod2, activation="relu"))
                            model.add(Dense(nod3, activation="relu"))
                            model.add(Dense(nod4, activation="relu"))
                            model.add(Dense(1, activation="sigmoid"))
                            model.compile(
                                loss=f1_loss,
                                optimizer="adam",
                                metrics=[
                                    f1,
                                    keras.metrics.Precision(name="precision"),
                                    keras.metrics.Recall(name="recall"),
                                    keras.metrics.AUC(name="auc"),
                                    keras.metrics.AUC(name="prc", curve="PR"),],)
                            return model

model = prepare_model()
history = (
    model.fit(
        trainX,
        trainy,
        callbacks=[Metrics(validation=(validateX, validatey))],
        epochs=150,
        batch_size=100,
        verbose=1,),)
tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience=20,
    mode="max",
    restore_best_weights=True,),
test_results = model.evaluate(testX, testy, verbose=0)

validation shape 115123
Epoch 1/150
6213/6213 [==============================] - 11s 1ms/step - loss: 0.2364 - f1: 0.7668 - precision: 0.6409 - recall: 0.8720 - auc: 0.7507 - prc: 0.7081
 — val_f1: 0.3726526611182655 — val_precision: 0.2396242738845631, — val_recall: 0.8377117179398548
Epoch 2/150
6213/6213 [==============================] - 8s 1ms/step - loss: 0.1893 - f1: 0.8111 - precision: 0.7336 - recall: 0.8706 - auc: 0.8102 - prc: 0.7520A: 2s - loss: 0.1896 - f1: 0.8109 - precision: 0.7324 - recall: 0.8702 - auc: 0.8099 -  - ETA: 1s - loss: 0.1895 - f1: 0.8109 - precision: 0.7327 - recall: 0.8703 - auc: 0.8100 - prc - ETA: 1s - loss: 0.1895 - f1: 0.8110 - precision: 0.7329 - recall: 0.8
 — val_f1: 0.3690245661996431 — val_precision: 0.23646051287041994, — val_recall: 0.8398721050812306
Epoch 3/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1862 - f1: 0.8139 - precision: 0.7441 - recall: 0.8749 - auc: 0.8120 - prc: 0.7530
 — val_f1: 0.38164646686500403 — v

6213/6213 [==============================] - 8s 1ms/step - loss: 0.1808 - f1: 0.8192 - precision: 0.7596 - recall: 0.8828 - auc: 0.8119 - prc: 0.7522A: 8s - loss: 0.1815 - f1: 0.8188 - precis
 — val_f1: 0.3954370964927587 — val_precision: 0.26447076671723657, — val_recall: 0.7833563774628414
Epoch 14/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1839 - f1: 0.8161 - precision: 0.7602 - recall: 0.8827 - auc: 0.8118 - prc: 0.7522
 — val_f1: 0.3796484515126151 — val_precision: 0.24672330725793917, — val_recall: 0.8231075008641549
Epoch 15/150
6213/6213 [==============================] - 8s 1ms/step - loss: 0.1814 - f1: 0.8186 - precision: 0.7607 - recall: 0.8824 - auc: 0.8116 - prc: 0.7521A: 7s - loss: 0.1814 - f1: 0.8187 - precision: 0 - ETA: 4s - loss:
 — val_f1: 0.39809536010932467 — val_precision: 0.26437140183205243, — val_recall: 0.8055651572761839
Epoch 16/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1821 - f1: 0.8179 - precision: 0

6213/6213 [==============================] - 8s 1ms/step - loss: 0.1792 - f1: 0.8208 - precision: 0.7623 - recall: 0.8829 - auc: 0.8091 - prc: 0.7496A: 7s - loss: 0.1772 - f1: 0.8229 - precision: 0.7622 - recall: 0.883
 — val_f1: 0.3940630953128598 — val_precision: 0.26002154874713374, — val_recall: 0.8133425509851365
Epoch 32/150
6213/6213 [==============================] - 8s 1ms/step - loss: 0.1797 - f1: 0.8203 - precision: 0.7626 - recall: 0.8828 - auc: 0.8091 - prc: 0.7497
 — val_f1: 0.3821937435325957 — val_precision: 0.24826783867631852, — val_recall: 0.8298479087452472
Epoch 33/150
6213/6213 [==============================] - 8s 1ms/step - loss: 0.1802 - f1: 0.8198 - precision: 0.7625 - recall: 0.8830 - auc: 0.8091 - prc: 0.7496A: 7s - loss: 0.1774 - f1: 0.8225 - precision: 0. - ETA: 4s - loss: 0.1796 - f1: 0 - ETA: 0s - loss: 0.1802 - f1: 0.8198 - precision: 0.7625 - recall: 0.8830 - auc: 0.
 — val_f1: 0.38903757183460924 — val_precision: 0.25597204400884543, — val_recall: 0.8

6213/6213 [==============================] - 9s 1ms/step - loss: 0.1848 - f1: 0.8152 - precision: 0.7626 - recall: 0.8823 - auc: 0.8078 - prc: 0.7484A: 3s - loss: 0.1850 - f1: 0.8150 - precision: 0.7626 - recall: 0.8823 - auc: 0.8078 - p - ETA: 2s - loss: 0.1850 - f1: 0.8150 - precision: 0
 — val_f1: 0.37766573149437915 — val_precision: 0.24429554049595167, — val_recall: 0.8317490494296578
Epoch 45/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1842 - f1: 0.8158 - precision: 0.7626 - recall: 0.8824 - auc: 0.8077 - prc: 0.7483A: 5s - loss: 0.1843 - f1: 0.8158 - precision: 0.7626 - recall: 0.8824 - auc: 0.8077 - prc: 0 - ETA: 5s - loss: 0.1843 - f1: 0.8157 - precision: 0.7626 - recall: 0.8824 - auc: 0 - ETA: 4s - loss: 0.1844 - f1: 0.8156 - precision: 0.7626 - recall: 0.8824 - auc: 0.8077 - prc: - ETA: 4s - loss: 0.1844 - f1: 0.8156 - precision: 0.7626 - recall: 0.8824 - auc: 0.8077 -  - ETA: 3s - loss: 0.1844 - f1: 0.8156 - precision: 0.7626 - recall: 0.8824 - au

6213/6213 [==============================] - 9s 1ms/step - loss: 0.1888 - f1: 0.8112 - precision: 0.7629 - recall: 0.8807 - auc: 0.8063 - prc: 0.7473A: 7s - loss: 0.1887 - f1: 0.8112 - precision: 0.7629 - recall: 0.8808 - auc: 0.8063 - prc: 0. - ETA: 7s - loss: 0.1889 - f1: 0.8111 - precision: 0.7629 - recall: 0.8808 - auc: 0.8063 - prc: 0.747 - ETA: 7s - loss: 0.1889 - f1: 0.8111 - precision: 0.7629 - recall: 0.8808 - auc: 0.8063 - ETA: 6s - loss: 0.1885 - f1: 0.8115 - precision: 0.7629 - recall: 0.8808 - auc: 0.8063 - prc:  - ETA: 6s - loss: 0.1887 - f1: 0.8113 - precision: 0.7629 - recall: 0.8808 - auc: 0.8063 - prc: 0.747 - ETA: 6s - loss: 0.1887 - f1: 0.8112 - precision: 0.7629 - recall: 0.88 - ETA: 4s - loss: 0.1893 - f1: 0.8107 - precision: 0.7629 - recall: 0.8807 - auc: 0.8063 - prc: - ETA: 4s - loss: 0.1892 - f1: 0.8108 - precision: 0.7629 - recall: 0.8807 - auc: - ETA: 3s - loss: 0.1891 - f1: 0.8109 - precision: 0.7629 - recall: 0.8807 - auc: 0.8063 - ETA: 2s - loss: 0.1891 -

...

6213/6213 [==============================] - 8s 1ms/step - loss: 0.1879 - f1: 0.8121 - precision: 0.7626 - recall: 0.8773 - auc: 0.8040 - prc: 0.7456A: 7s - loss: 0.1853 - f1: 0.8146 - precision: 0.7626 - recall: - ETA: 5s - loss: 0.1857 - f1: 0.8143 - precision: 0.7626 - recall: 0.8773 - auc: 0.8040 - prc: 0. - ETA: 5s - loss: 0.1860 - f1: 0.8140 - precision: 0.7626 - recall: 0.8773 - auc: 0.8040 - prc: - ETA: 5s - loss: 0.1865 - f1: 0.81 - ETA: 1s - loss: 0.1877 - f1: 0.8123 - precision: 0.7626 - recall: 0.8773 - auc: 0.8040 - - ETA: 1s - loss: 0.1878 - f1: 0.8122 - precision: 0.7626 - recall: 0.8773 - auc
 — val_f1: 0.3646583654400952 — val_precision: 0.2323805005337445, — val_recall: 0.8465260974766678
Epoch 99/150
6213/6213 [==============================] - ETA: 0s - loss: 0.1876 - f1: 0.8124 - precision: 0.7626 - recall: 0.8772 - auc: 0.8040 - prc: 0.7456- ETA: 7s - loss: 0.1874 - f1: 0.8125 - precision: 0.7627 - recall: 0.8772 - au - ETA: 6s - loss: 0.1875 - f1: 0.8125 - precis

6213/6213 [==============================] - 8s 1ms/step - loss: 0.1872 - f1: 0.8127 - precision: 0.7629 - recall: 0.8762 - auc: 0.8037 - prc: 0.7455A: 8s - loss: 0.1872 - f1: 0.8128 - precision: 0.7629 - recall: 0.8763 - auc: 0.8038 - prc: 0. - ETA: 7s - loss: 0.1867 - f1: 0.8132 - precision: 0.7629 -  - ETA: 5s - loss: 0.1861 - f1: 0.8138 - precision: 0.7629 - recall: 0.8762 - auc: - ETA: 4s - loss: 0.1864 - f1: 0.8136 - precision: 0.7629 - recall: 0.8762 - auc: 0.8 - ETA: 3s - loss: 0.1867 - f1: 0.8132 - precision: 0.7629 - recall: 0.8762 - auc: 0.8037 - prc: 0.745 - ETA: 3s - loss: 0.1868 - f1: 0.8132 - precision: 0.7629 - recall: 0.8762 - auc: 0.8037 - prc: 0 - ETA: 3s - loss: 0.1868 - f1: 0.8131 - precision: 0.7629 - recall: 0.8762 - auc: 0.8037 - prc: 0.74 - ETA: 3s - loss: 0.1869 - f1: 0.8131 - precision: 0.7629 - recall: 0.8762 - au - ETA: 2s - loss: 0.1872 - f1: 0.8128 - precision: 0.7629 - recall:  - ETA: 0s - loss: 0.1872 - f1: 0.8127 - precision: 0.7629 - recall: 0.8762 - 

6213/6213 [==============================] - 9s 1ms/step - loss: 0.1843 - f1: 0.8157 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - prc: 0.7452A: 7s - loss: 0.1842 - f1: 0.8158 - precision: 0.7627 - - ETA: 5s - loss: 0.1838 - f1: 0.8162 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - prc: 0.74 - ETA: 4s - loss: 0.1838 - f1: 0.8162 - precision: 0.7627 - recall: 0.8763 - auc: 0 - ETA: 3s - loss: 0.1840 - f1: 0.8160 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - prc: 0.745 - ETA: 3s - loss: 0.1840 - f1: 0.8160 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - prc - ETA: 3s - loss: 0.1840 - f1: 0.8160 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - ETA: 2s - loss: 0.1841 - f1: 0.8159 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - - ETA: 2s - loss: 0.1841 - f1: 0.8159 - precision: 0.7627 - recall: 0.8763 - auc: 0.8035 - prc: 0.745 - ETA: 2s - loss: 0.1841 - f1: 0.8159 - precision: 0.7627 - reca
 — val_f1: 0.39655966825372446 — val_precision: 0.2657550360241141

6213/6213 [==============================] - 9s 1ms/step - loss: 0.1877 - f1: 0.8122 - precision: 0.7630 - recall: 0.8761 - auc: 0.8035 - prc: 0.7453A: 6s - loss: 0.1916 - f1: 0.8084 - precision: 0.7630 - recall: 0.8761 - auc: 0.8035 - prc: 0.7 - ETA: 6s - loss: 0.1915 - f1: 0.8085 - precision: 0.7630 - recall: 0.8761 - auc: 0.8035 - ETA: 5s - loss: 0.1908 - f1: 0.8092 - precision: 0.7630 - recall: 0.8761 - auc: 0.8035 - prc - ETA: 5s - lo - ETA: 0s - loss: 0.1879 - f1: 0.8120 - precision: 0.7630 - recall: 0.8761 - auc: 0.8035 - 
 — val_f1: 0.3556536322711625 — val_precision: 0.2234819071284361, — val_recall: 0.870463187003111
Epoch 131/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1845 - f1: 0.8155 - precision: 0.7630 - recall: 0.8762 - auc: 0.8035 - prc: 0.7453A: 6s - loss: 0.1894 - f1: 0.8106 - precision: 0.7630 - rec - ETA: 4s - loss: 0.1869 - f1: 0.8131 - precision: 0.7630 - recall: 0.8762 - auc: 0.8035 - p - ETA: 4s - loss: 0.1864 - f1: 0.8136 - precision

6213/6213 [==============================] - 9s 2ms/step - loss: 0.1847 - f1: 0.8153 - precision: 0.7631 - recall: 0.8766 - auc: 0.8037 - prc: 0.7454A: 9s - loss: 0.1859 - f1:
 — val_f1: 0.3738092505451624 — val_precision: 0.24003832358866015, — val_recall: 0.8443657103352921
Epoch 141/150
6213/6213 [==============================] - 10s 2ms/step - loss: 0.1850 - f1: 0.8150 - precision: 0.7630 - recall: 0.8767 - auc: 0.8037 - prc: 0.7454: 4s - loss: 0.1842 - f1: 0.8158 - precision: 0.7630 - recall: 0.8767 -  - ETA: 2s - loss: 0.1846 - f1: 0.8154 - precision: 0.7630 - reca
 — val_f1: 0.38556027044037927 — val_precision: 0.25198375839282394, — val_recall: 0.820515036294504
Epoch 142/150
6213/6213 [==============================] - 9s 1ms/step - loss: 0.1842 - f1: 0.8158 - precision: 0.7630 - recall: 0.8766 - auc: 0.8037 - prc: 0.7454A: 8s - loss: 0.1828 - f1: 0.8170 - precision: 0.7631 - recall: 0.8766 - auc: 0.8037 - prc: 0.74 - ETA: 8s - loss: 0.1838 - f1: 0.8161 - precision: 0.7631 - 

## Results

In [42]:
test_results = model.evaluate(testX, testy, verbose=0)
for x, y in zip(model.metrics_names, test_results):
  print(x, ': ', y)
print()

loss :  0.6198608875274658
f1 :  0.3801007568836212
precision :  0.7631115317344666
recall :  0.8763002157211304
auc :  0.803955078125
prc :  0.7452502250671387



## Feature importance

In [ ]:
trainX_raw, validateX_raw, testX_raw, trainy_raw, validatey_raw, testy_raw = split_dataset(df,sample=1)

In [ ]:
my_model = KerasRegressor(build_fn=prepare_model)    
my_model.fit(trainX,trainy)

perm = PermutationImportance(my_model, random_state=1).fit(trainX,trainy)
eli5.show_weights(perm, feature_names = trainX_raw.columns.tolist())

In [62]:
eli5.show_weights(perm, feature_names = trainX_raw.columns.tolist())

Weight,Feature
0.1126 ± 0.0012,combined_followers
0.0313 ± 0.0003,loudness
0.0258 ± 0.0007,danceability
0.0149 ± 0.0004,valence
0.0140 ± 0.0004,acousticness
0.0093 ± 0.0006,duration_ms
0.0082 ± 0.0004,energy
0.0063 ± 0.0003,release_day
0.0044 ± 0.0003,explicit
0.0025 ± 0.0002,speechiness


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=efdb1931-d19c-4850-b12b-726a7087f8c0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>